In [2]:
import pandas as pd
pd.options.display.max_columns = 999
import pgeocode as geo
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [3]:
sc= pd.read_excel(r'Input\\social capital variables spreadsheet.12-8-17.xlsx', sheet_name='Sheet1')
sc.head(1)

,FIPS,County_Name,Religious2014,Civic2014,Business2014,Political2014,Professional2014,Labor2014,Bowling2014,Recreational2014,Golf2014,Sports2014,pop2014,assn2014,pvote2012,respn2010,nccs2014,sk2014
0,1001,"Autauga County, AL",53,7,3,0,1,4,1,5,2,0,55290,1.37457,0.644956,0.78,157,-0.631003


In [4]:
cols_rename= {
            'County_Name':'area',
            'Recreational2014': 'fit2014',
            'Religious2014': 'relig2014',
            'Religious2014': 'relig2014',
            'Business2014': 'bus2014',
            'Political2014': 'pol2014',
            'Professional2014': 'prof2014',
            'Bowling2014': 'bowl2014',
            'Sports2014': 'sport2014'
            }

In [5]:
sc.rename(columns=cols_rename, inplace=True)
sc.columns= sc.columns.str.strip().str.lower()
sc.columns=sc.columns.str.replace('\d+','',regex=True).str.strip()
sc.loc[:,'area'].replace(regex=True, to_replace=" County", value= r"", inplace= True)
sc['nccs']= sc['nccs'].replace(np.nan, 0) # This is one of the key things that make results: Before this fix: corr= 0.54. After this fix corr= 0.99

sc['nccs']= sc['nccs']/(sc['pop']/1000)
sc['assn']= (sc['relig'] + sc['civic'] + sc['bus'] + sc['pol'] + sc['prof'] + sc['labor'] + sc['bowl'] + sc['fit'] + sc['golf'] + sc['sport']) / (sc['pop']/1000)

sc.drop(columns=['bowl', 'civic', 'fit', 'golf', 'relig', 'sport', 'pol', 'prof', 'bus', 'labor'], inplace=True)
sc.drop(columns=['pop'],inplace=True)
sc.head(3)

,fips,area,assn,pvote,respn,nccs,sk
0,1001,"Autauga, AL",1.374570,0.644956,0.78,2.839573,-0.631003
1,1003,"Baldwin, AL",1.156660,0.674735,0.73,3.595159,-0.555396
2,1005,"Barbour, AL",0.820436,0.665031,0.63,3.430916,-0.891036


In [6]:
x = sc.loc[:, ['respn','pvote','assn','nccs']].values
# Separating out the target
y = sc.loc[:,['fips']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

pca = PCA(n_components=1)
principalComponents = pca.fit_transform(x)
pca_df = pd.DataFrame(data = principalComponents
             , columns = ['pc1'])
pca_df.head(2)

,pc1
0,-0.631075
1,-0.555699


In [7]:
sc['pc1']=pca_df[['pc1']]

In [8]:
sc.head()

,fips,area,assn,pvote,respn,nccs,sk,pc1
0,1001,"Autauga, AL",1.374570,0.644956,0.78,2.839573,-0.631003,-0.631075
1,1003,"Baldwin, AL",1.156660,0.674735,0.73,3.595159,-0.555396,-0.555699
2,1005,"Barbour, AL",0.820436,0.665031,0.63,3.430916,-0.891036,-0.890722
3,1007,"Bibb, AL",1.020001,0.656838,0.58,2.394785,-0.906582,-0.906795
4,1009,"Blount, AL",0.849839,0.708668,0.80,1.873114,-1.013280,-1.013599


In [9]:
sc['sk'].corr(sc['pc1'])

0.9999999868272843

In [10]:
sc.shape

(3141, 8)

In [11]:
sc['year']=2014
sc.to_csv("output\\sc14.csv")